# Zone de couverture du metro toulousain (avec la future ligne 3)

Application de [OAlley API](https://api.oalley.fr) pour creer une carte de couverture des metros toulousains, avec la future ligne.

In [40]:
import folium
import grequests
import requests
import polyline
from urllib.parse import urlencode
import json
from shapely.geometry.polygon import Polygon
from shapely.ops import cascaded_union

baseurl = "https://api.oalley.fr/api/AppKeys/"
method = "/isochronous?"

appkey = "YOUR-API-KEY"


## Get the list of stations


In [7]:
# Request bike stations
r = requests.get("https://data.toulouse-metropole.fr/explore/dataset/stations-de-metro/download/?format=json&timezone=Europe/Berlin")

print(r.json[0])

{'recordid': '936c4b78a37c8ef50a42849a117c4501d0b03cdf', 'fields': {'geo_shape': {'type': 'Point', 'coordinates': [1.459262812511472, 43.579584532680684]}, 'nom': 'SAOUZELONG', 'etat': 'Existant', 'ligne': 'B', 'geo_point_2d': [43.579584532680684, 1.459262812511472]}, 'geometry': {'type': 'Point', 'coordinates': [1.459262812511472, 43.579584532680684]}, 'datasetid': 'stations-de-metro', 'record_timestamp': '2016-06-26T07:01:01+02:00'}


In [22]:
# Extract coords for each
coords = [{'lat': s['fields']['geo_shape']['coordinates'][1], 'lng': s['fields']['geo_shape']['coordinates'][0], 'duration': 0} for s in r.json()]

coords_5m = [dict(c, duration = 120) for c in coords]

print("Found %d stations" % len(coords))

zones = []

i = 0

# Called when OAlley returned a zone
def callback(res, **kwargs):
    global i
    if(res.status_code != 200): 
        return print(res.json()) 
    
    body = res.json()
    zone = polyline.decode(body['polyline'])
    zones.append(zone)
    i += 1
    if(i % 10 == 0):
        print("%d / %d" % (i, len(coords)))
    
def exception_handler(request, exception):
    print("Error : %s" % exception)
    
    
# Prepare all requests for OAlley
urls = [baseurl + appkey + method + urlencode(point) for point in coords_5m]
reqs = [grequests.get(url, callback=callback) for url in urls]

# Execute all requests
grequests.map(reqs, exception_handler=exception_handler)

print("Done !")

Found 44 stations
10 / 44
20 / 44
30 / 44
40 / 44
Done !


# Carte réaliste des stations de la ligne 3 du métro Toulousain

Pour d'autres visualisations plus poussées, voir sur le dépot [Github](https://github.com/Overdrivr/interactive-geospatial-analysis)

In [39]:
# Build output map
m = folium.Map(location=[46, 2], zoom_start=5)

# Source: http://actu.cotetoulouse.fr/files/2015/12/11232984_933782070010705_6978152120249447133_n.jpg
# Source: http://actu.cotetoulouse.fr/toulouse-tout-ce-qu-il-faut-savoir-sur-le-trace-de-la-3e-ligne-de-metro-et-la-facon-dont-il-pourrait-evoluer_25984/
# Tool to get GPS coords : http://www.gps-coordinates.net/
ligne3locs = [
    [43.6164113,1.3505030],# Airbus colommiers
    [43.6119056,1.3726646],# Aibus Saint Martin
    [43.6217381,1.3969560],# Jean Maga
    [43.6164200,1.4076825],# Espace Job
    [43.6170028,1.4220444],# Boulevard de suisse
    [43.6294087,1.4294307],# Fondeyre (estimation from previews)
    [43.6346464,1.4398956],# La Vache SNCF
    [43.6270672,1.4443802],# Toulouse Lautrec (estimation from previews)
    [43.6168462,1.4508390],# Raynal
    [43.6176851,1.4598298],# Bonnefoy (estimated from previews)
    [43.6107874,1.4550018],# Marengo
    [43.6006259,1.4520406],# Francois verdier
    [43.5958397,1.4631342],# Jean rieux
    [43.5921460,1.4691190],# Cote pavee
    [43.5786969,1.4842483],# L'Ormeau
    [43.5734577,1.4805364],# Montaudran
    [43.5600868,1.4983034]# Airbus Defence and space
]

for pt in ligne3locs:
    print(pt)
    folium.Marker(pt).add_to(m)

    
m.fit_bounds(m.get_bounds())
# IPython trick to display the map
m

[43.6164113, 1.350503]
[43.6119056, 1.3726646]
[43.6217381, 1.396956]
[43.61642, 1.4076825]
[43.6170028, 1.4220444]
[43.6294087, 1.4294307]
[43.6346464, 1.4398956]
[43.6270672, 1.4443802]
[43.6168462, 1.450839]
[43.6176851, 1.4598298]
[43.6107874, 1.4550018]
[43.6006259, 1.4520406]
[43.5958397, 1.4631342]
[43.592146, 1.469119]
[43.5786969, 1.4842483]
[43.5734577, 1.4805364]
[43.5600868, 1.4983034]


In [6]:
# Eventually, dump all zones to a google polyline format if you want to display it on a website
arr = [ polyline.encode(list(z.exterior.coords)) for z in allzones]

json.dumps(arr)

'["ehdiGuhpGcAiCcAbD]nCdAfEQ@M?eLy@oLxBCDhBkPScGhB`A\\\\E|GTfHqEwCq@kGTlG_JBwEHoFcPjPu@aAeAqCmBnI]AeEu@uB`AUHz@fCFnAuGdClCdD@?_DzL|B|B~EGbBxFhEp@PyAxON|BN|@d@jC_Ad@}B|@zC|@zGtEdJv@gGtJvHfDwDjF}@cPgMhDyBqBeCZqAgDmBk@aATgCdDcQ{IjM}Ai@yBhC", "wkfiGs~oGi@mCqU}n@~Ixe@uK~@|EtB[xAwHJ`AlIi@xDoCjDcBtJf_@aT?N`AmAXAZA\\\\?X?r@J@@D?HCHQjLQqLi@vBcBC@kCnA_@By@y@ESEO", "{meiG}fsG{GuMaB~FyAnCdAbIkAHzEdGFj@qIjE_JrMVdFpHwBxGdCj@vCrHmBfBwBbCoCn@_DvBoAn@s@oA{DQ]fI_Cu@yA^uE`@mD{C{BuAk@wEx@uCUmBO", "ax_iGqplG{@pAiBjC_@n@a@j@yVaJXxC}C~HaAZ@@JHRLZP??VP`AbIB`DnLgDd@dKPAxACfJlCxEdDtBxAnChB`KARsJbG}@dAXcDwQj@mB`@{@`Au@vA}@f@aAlBc@jGcD|GcEeOMlA_Iw@UqCN}CS\\\\eIkAGxAkGiBkMRs@x@AzBnMvHuQn@VvQfOiEiSoCqJpD_Dc@kB[wBfPgGoD}EkCaCwBp@qQqEd@aHkBhGw`@_KnCoADAti@}Rt@j@qBzAzCZb@j@h@?}ArJ?jTxF{B`Fq@b@aHp@yIdCiBXe@vDt@_C_GFoBcGk@lBsBnAwCeDHUQMe@dDeTcGxEiD|AoBkEHuAKC_@Mi@iAx@eCzAqHNOpAkH|FyAzAd@lA\\\\hAwAqA{DlBaCB@MGeC_B@Yn@c@f@yCf@kAlHpGgBlCYPvHjBGFmElE`AvAfAzAnBb@bCaBrAlCjD~FfCe@{AxC`A~AzBhAlBsF|ElBRv@?CJq@`GzG?eG{AcE\\\\mAqA